In [1]:
!source /usr/local/Ascend/ascend-toolkit/set_env.sh

In [44]:
import os
import re
import numpy as np
import pandas as pd
from IPython.display import display

In [3]:
DATA_DIR = "../data/mimic-4"
TABLE = "diagnoses_icd.csv"
GEM = "2018_I9gem.txt"

In [25]:
df = pd.read_csv(os.path.join(DATA_DIR, TABLE))

In [26]:
df.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


In [45]:
df.shape

(6364488, 6)

### ICD-9 to ICD-10 conversion

In [31]:
def read_gem(path):
    with open(path, "r") as f:
        lines = f.readlines()

        def processor(s):
            s = [x for x in s.strip().split(" ") if x != ""]
            return {
                "icd9": s[0],
                "icd10": s[1],
                "flags": s[2]
            }

    lines = [processor(line) for line in lines]
    gem = pd.DataFrame.from_records(lines)
    return gem

In [32]:
gem = read_gem(os.path.join(DATA_DIR, GEM))

In [33]:
gem.head()

,icd9,icd10,flags
0,0010,A000,00000
1,0011,A001,00000
2,0019,A009,00000
3,0020,A0100,10000
4,0021,A011,00000


In [46]:
gem.shape

(24860, 5)

In [50]:
def norm(code: str) -> str:
    if pd.isna(code):
        return None
    code = re.sub(r'[\s\.]', '', str(code).upper())
    if code.isdigit() and len(code) < 3:
        code = code.zfill(3)
    return code
    
def icd9_to_icd10_fuzzy(icd9: str, mapping: dict) -> str:
    if icd9 in mapping:
        return mapping[icd9]

    code = icd9
    while len(code) < 5:
        code += "0"
        if code in mapping:
            return mapping[code]

    desc = {k: v for k, v in mapping.items() if k.startswith(icd9)}
    if desc:
        for k in sorted(desc):
            if k.endswith("0"):
                return desc[k]
        return next(iter(desc.values()))
    return None

def icd9_to_icd10(
    df: pd.DataFrame,
    gem: pd.DataFrame,
    df_icd_code_col:str="icd_code",
    df_icd_version_col:str="icd_version", 
    gem_icd9_col="icd9", 
    gem_icd10_col="icd10"
) -> pd.DataFrame:
    '''
    converts dataframe with 
    '''
    print("Normalization...")
    gem["icd9_norm"]  = gem[gem_icd9_col].apply(norm)
    gem["icd10_norm"] = gem[gem_icd10_col].apply(norm)
    gem["flags"]      = gem["flags"].astype(str)
    df["icd_code_norm"] = df[df_icd_code_col].apply(norm)
    print("ICD codes are normalized")

    print("Number of bad mappings:", gem["flags"].str[0].eq("7").sum(), "out of", len(gem), "let's through out them")
    gem_ok = gem[~gem["flags"].str[0].eq("7")]  
    gem_ok["rank"] = gem_ok["flags"].str[0].astype(int)   
    gem_ok = gem_ok.sort_values(["icd9_norm", "rank"])
    MAP_9to10 = gem_ok.groupby("icd9_norm")["icd10_norm"].first().to_dict()
    print("Mapping 9to10 is created")
    
    def map_row(row):
        if row[df_icd_version_col] == 10:
            return row["icd_code_norm"]
        return icd9_to_icd10_fuzzy(row["icd_code_norm"], MAP_9to10)
    
    print("Mapping started...")
    df["icd10_code"] = df.apply(map_row, axis=1)
    df.drop(columns="icd_code_norm", inplace=True)
    print("Mapping finished")

    n9  = (df[df_icd_version_col] == 9).sum()
    nun = df["icd10_code"].isna() & (df[df_icd_version_col] == 9)
    print(f"Не смаппилось ICD-9 кодов: {nun.sum()} из {n9}")

    return df

In [51]:
df = icd9_to_icd10(df, gem)

Normalization...
ICD codes are normalized
Number of bad mappings: 0 out of 24860 let's through out them
Mapping 9to10 is created
Mapping started...
Mapping finished
Не смаппилось ICD-9 кодов: 0 из 2908741


### Truncation to category

In [54]:
df["icd10_category"] = df["icd10_code"].str[:3]

### Save

In [56]:
df.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,icd10_code,icd10_category
0,10000032,22595853,1,5723,9,K766,K76
1,10000032,22595853,2,78959,9,R188,R18
2,10000032,22595853,3,5715,9,K740,K74
3,10000032,22595853,4,07070,9,B1920,B19
4,10000032,22595853,5,496,9,J449,J44


In [57]:
df.to_csv("diagnoses_icd10.csv")